# Important libraries

In [101]:
import win32com.client
import sys
import time
import os
import numpy as np
import plotly.express as px
import pandas as pd

# Openserver Connection

Openserver class with methods to connect to the server, send commands, and disconnect from the server.

In [102]:
class OpenServer():
    "Class for holding ActiveX reference. Allows license disconnection"
    def __init__(self):
        self.status = "Disconnected"
        self.OSReference = None
    
    def Connect(self):
        self.OSReference = win32com.client.Dispatch("PX32.OpenServer.1")
        self.status = "Connected"
        print("OpenServer connected")
        
    def Disconnect(self):
        self.OSReference = None
        self.status = "Disconnected"
        print("OpenServer disconnected")

In [103]:
def GetAppName(sv):
    # function for returning app name from tag string
    pos = sv.find(".")
    if pos < 2:
        sys.exit("GetAppName: Badly formed tag string")
    app_name = sv[:pos]
    if app_name.lower() not in ["prosper", "mbal", "gap", "pvt", "resolve",
                                   "reveal"]:
        sys.exit("GetAppName: Unrecognised application name in tag string")
    return app_name


def DoCmd(OpenServe, cmd):
    # perform a command and check for errors
    lerr = OpenServe.OSReference.DoCommand(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoCmd: " + err)


def DoSet(OpenServe, sv, val):
    # set a value and check for errors
    lerr = OpenServe.OSReference.SetValue(sv, val)
    app_name = GetAppName(sv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSet: " + err)
    
def DoGet(OpenServe, gv):
    # get a value and check for errors
    get_value = OpenServe.OSReference.GetValue(gv)
    app_name = GetAppName(gv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGet: " + err)
    return get_value


def DoSlowCmd(OpenServe, cmd):
    # perform a command then wait for command to exit and check for errors
    step = 0.001
    app_name = GetAppName(cmd)
    lerr = OpenServe.OSReference.DoCommandAsync(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)
    while OpenServe.OSReference.IsBusy(app_name) > 0:
        if step < 2:
            step = step*2
        time.sleep(step)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)


def DoGAPFunc(OpenServe, gv):
    DoSlowCmd(gv)
    DoGAPFunc = DoGet(OpenServe, "GAP.LASTCMDRET")
    lerr = OpenServe.OSReference.GetLastError("GAP")
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGAPFunc: " + err)
    return DoGAPFunc


def OSOpenFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.OPENFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSOpenFile: " + err)


def OSSaveFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.SAVEFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSSaveFile: " + err)

# Single Well calculation

In [104]:
#gas availability and increment steps
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
oil_rate_array = []

In [105]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()

# Perform functions

cwd = os.getcwd()
OSOpenFile(petex, cwd + r'\well_1.Out', 'PROSPER')
for i in range(0, len(gas_injection_array)):
    command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
    DoSet(petex, command,\
        gas_injection_array[i])
DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
for i in range(0, len(gas_injection_array)):
    value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
    oil_rate_array.append(np.round(float((DoGet(petex, value ))),2))
OSSaveFile(petex, cwd + r'\well_1.Out', 'PROSPER')


#creating dataframe for gas injection rate and oil rate values
df = pd.DataFrame({'oil_rate':oil_rate_array, 'gas_injection':gas_injection_array})
df.head()

OpenServer connected


,oil_rate,gas_injection
0,1749.18,0.0
1,1767.97,0.2
2,1781.27,0.4
3,1790.62,0.6
4,1796.92,0.8


In [106]:
fig = px.line(df, x='gas_injection', y='oil_rate', markers=True)
fig.update_layout(title='Sensitivity Plot',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   xaxis={
                       'range':[0, df['gas_injection'].max()+0.2]
                   })
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()